### Load and Analyze Grant Data
The `download_data.py` script has already downloaded, extracted, and processed the raw XML data into a clean CSV file located at `data/parsed_grants.csv`. This notebook will now focus on loading this pre-processed data and performing an in-depth analysis of the grant information.

In [16]:
import pandas as pd

# Load the parsed data from the CSV file
grants_df = pd.read_csv('data/parsed_grants.csv')

# Display basic information and the first few rows to verify the data
print("Successfully loaded 'data/parsed_grants.csv'")
print(f"The dataset contains {grants_df.shape[0]} rows and {grants_df.shape[1]} columns.")
print("\nFirst 5 rows of the dataset:")
print(grants_df.head())

# Display summary statistics for numerical columns
print("\nSummary statistics for numerical columns:")
print(grants_df.describe())

# Check for missing values in each column
print("\nMissing values per column:")
print(grants_df.isnull().sum())

Successfully loaded 'data/parsed_grants.csv'
The dataset contains 327376 rows and 11 columns.

First 5 rows of the dataset:
    FilerEIN                    FilerName  ReturnType TaxPeriodEnd  \
0  208578192           OAKLAWN FOUNDATION         990   2024-06-30   
1  208578192           OAKLAWN FOUNDATION         990   2024-06-30   
2  570314406  North Greenville University         990   2024-05-31   
3  570314406  North Greenville University         990   2024-05-31   
4   43102943     NEW AMERICAN ASSOCIATION         990   2024-06-30   

   TotalGrantsPaid                  RecipientName  RecipientCity  \
0                0  ARKANSAS COMMUNITY FOUNDATION            NaN   
1                0        OAKLAWN CENTER ON AGING            NaN   
2                0      First Presbyterian Church            NaN   
3                0     Tigerville Fire Department            NaN   
4                0   CONGOLESE DEVELOPMENT CENTER            NaN   

   RecipientState  RecipientZIP  GrantAmount  

In [17]:
# Load the parsed data from the CSV file
grants_df = pd.read_csv('data/parsed_grants.csv')

# Display basic information and the first few rows to verify the data
print("Successfully loaded 'data/parsed_grants.csv'")
print(f"The dataset contains {grants_df.shape[0]} rows and {grants_df.shape[1]} columns.")
print("\nFirst 5 rows of the dataset:")
print(grants_df.head())

# Display summary statistics for numerical columns
print("\nSummary statistics for numerical columns:")
print(grants_df.describe())

# Check for missing values in each column
print("\nMissing values per column:")
print(grants_df.isnull().sum())

Successfully loaded 'data/parsed_grants.csv'
The dataset contains 327376 rows and 11 columns.

First 5 rows of the dataset:
    FilerEIN                    FilerName  ReturnType TaxPeriodEnd  \
0  208578192           OAKLAWN FOUNDATION         990   2024-06-30   
1  208578192           OAKLAWN FOUNDATION         990   2024-06-30   
2  570314406  North Greenville University         990   2024-05-31   
3  570314406  North Greenville University         990   2024-05-31   
4   43102943     NEW AMERICAN ASSOCIATION         990   2024-06-30   

   TotalGrantsPaid                  RecipientName  RecipientCity  \
0                0  ARKANSAS COMMUNITY FOUNDATION            NaN   
1                0        OAKLAWN CENTER ON AGING            NaN   
2                0      First Presbyterian Church            NaN   
3                0     Tigerville Fire Department            NaN   
4                0   CONGOLESE DEVELOPMENT CENTER            NaN   

   RecipientState  RecipientZIP  GrantAmount  

In [ ]:
# Define keywords for a broad search related to kidney health
keywords = [
    "kidney", "organ donation", "organ, "transplant", "dialysis",
    "smoking cessation", "tobacco", "weight loss", "obesity",
    "fitness", "lifestyle change", "behavior change", "public health",
    "preventive health", "underserved", "health equity", "minority health", "nephrology"
]

# Create a regex pattern to search for any of the keywords, case-insensitively
regex_pattern = '|'.join(keywords)

# Filter the DataFrame for grants where 'GrantPurpose' contains any of the keywords.
# The `na=False` argument ensures that NaN values in 'GrantPurpose' are treated as not matching.
kidney_related_grants_df = grants_df[grants_df['GrantPurpose'].str.contains(regex_pattern, case=False, na=False)].copy()

# Display the number of grants found and show a sample of the results
print(f"Found {len(kidney_related_grants_df)} grants related to the specified keywords.")
print("\nFirst 10 rows of the filtered dataset:")
# Calculate the average grant amount for each filer and sort them in descending order
print(kidney_related_grants_df.head(10))

Found 3196 grants related to the specified keywords.

First 10 rows of the filtered dataset:
      FilerEIN             FilerName  ReturnType TaxPeriodEnd  \
11   870800705  Fuse Innovation Fund         990   2024-06-30   
12   870800705  Fuse Innovation Fund         990   2024-06-30   
13   870800705  Fuse Innovation Fund         990   2024-06-30   
14   870800705  Fuse Innovation Fund         990   2024-06-30   
15   870800705  Fuse Innovation Fund         990   2024-06-30   
16   870800705  Fuse Innovation Fund         990   2024-06-30   
255  882722663      AGE WELL AT HOME         990   2024-12-31   
256  882722663      AGE WELL AT HOME         990   2024-12-31   
257  882722663      AGE WELL AT HOME         990   2024-12-31   
381  741159753     BAYLOR UNIVERSITY         990   2024-05-31   

     TotalGrantsPaid                               RecipientName  \
11                 0                      PA Alliance Foundation   
12                 0                           Justice 

In [ ]:
# Load the filer data, specifying dtype for 'EIN' to ensure consistency
filer_df = pd.read_csv('data/parsed_filer_data.csv', dtype={'EIN': str})

# Ensure 'FilerEIN' in grants_df is also a string to prevent merge issues
kidney_related_grants_df['FilerEIN'] = kidney_related_grants_df['FilerEIN'].astype(str)

# Merge the kidney-related grants with the filer data
# This join adds detailed filer information to each grant record
merged_df = pd.merge(kidney_related_grants_df, filer_df, left_on='FilerEIN', right_on='EIN', how='left')

# Calculate the average grant amount for each filer and add it as a new column
# The 'transform' function is used to broadcast the mean value back to the original DataFrame shape
merged_df['AverageGrantAmount'] = merged_df.groupby('FilerName')['GrantAmount'].transform('mean')

# Sort the DataFrame first by the average grant amount in descending order,
# then by the individual grant amount in descending order for filers with the same average
final_ranked_df = merged_df.sort_values(by=['AverageGrantAmount', 'GrantAmount'], ascending=[False, False])

# Display the top 20 grants from the ranked list, showing key information
print("Top 20 Kidney-Related Grants, Ranked by Filer's Average Grant Amount:")
print(final_ranked_df[[
    'FilerName', 'GrantAmount', 'AverageGrantAmount', 
    'RecipientName', 'GrantPurpose', 'Organization501cType', 'TotalRevenue'
]].head(20))

# Save the final ranked DataFrame to a new CSV file for further analysis or reporting
final_ranked_df.to_csv('data/kidney_related_grants_ranked.csv', index=False)

Top 20 Kidney-Related Grants, Ranked by Filer's Average Grant Amount:
                                        FilerName  GrantAmount  \
1529  The Kansas University Endowment Association    253382689   
1530  The Kansas University Endowment Association    253382689   
2607                          ANCHORUM ST VINCENT     50550000   
2608                          ANCHORUM ST VINCENT     50550000   
3461              CHRISTIAN COMMUNITY DEVELOPMENT     41156603   
1550                     FUTURE OF LIFE INSTITUTE     50006500   
1551                     FUTURE OF LIFE INSTITUTE     50006500   
1548                     FUTURE OF LIFE INSTITUTE     25025000   
1549                     FUTURE OF LIFE INSTITUTE     25025000   
34                             JEWISH UNITED FUND     36308755   
35                             JEWISH UNITED FUND     36308755   
2298                              NRG HOUSING INC     31355818   
423                      WILLOW CREEK HOUSING INC     27759784   
1547  